This document explains the data preparation process for training our matching model. The example data comes from a research project that digitized historic records of German joint-stock companies [(Gram et al. 2022)](https://dl.acm.org/doi/10.1145/3531533). The data contains inconsistencies in spelling, primarily due to variations in abbreviation conventions and OCR errors, across most variables. These challenges make it a compelling real-world use case for entity matching.

The data consists of three files:

- *left.csv*
- *right.csv*
- *matches.csv*

## Different Data Relationships

### 1. Loading the Data

First, we import the necessary libraries and load the datasets.


In [ ]:
import random
import pandas as pd

matches = pd.read_csv('matches.csv')
left = pd.read_csv('left.csv')
right = pd.read_csv('right.csv')

### 2. Inspecting the Data

Let's view the first few rows of each dataset to understand their structure.


In [ ]:
matches.head()

In [ ]:
left.head()

In [ ]:
right.head()

All three DataFrames have the same number of observations:


In [ ]:
print(f'Number of observations in matches: {len(matches)}')
print(f'Number of observations in left: {len(left)}')
print(f'Number of observations in right: {len(right)}')

### 3. Simulating a Many-to-Many Relationship

To demonstrate how to handle more complex matching scenarios, we simulate a many-to-many (m:m) relationship. For instance, assume that the company with `company_id` *1e87fc75b4* in the *left* DataFrame should match with two entries in the *right* DataFrame: the original match *0008e07878* and an additional match *8bf51ba8a0*.


In [ ]:
# Add an extra match to simulate a many-to-many relationship

extra_match = pd.DataFrame({
    'company_id_left' : ['1e87fc75b4'],
    'company_id_right' : ['8bf51ba8a0']
})
matches = pd.concat([matches, extra_match], ignore_index=True)

Now, inspect the modified `matches` dataframe for the affected IDs:


In [ ]:
matches[
    matches['company_id_left'].isin(['1e87fc75b4', '810c9c3435']) |
    matches['company_id_right'].isin(['0008e07878', '8bf51ba8a0'])
]

### 4. Understanding the Matching Issue

Simply adding a new row to the *matches* DataFrame can be problematic. Consider this simplified example:

| Left | Right | Implied Real-World Entity |
|------|-------|---------------------------|
| A    | C     | Entity 1                  |
| B    | D     | Entity 2                  |

If further evidence shows that record *A* and record *C* represent the same entity, then all related records (*A*, *B*, *C*, *D*) should be grouped together. This comprehensive grouping implies that every possible pair among these records should be represented (as shown in the first six rows of the table below). Notice that the observations *B* and *C* would consequently appear in both the *Left* and *Right* columns. Therefore, the *left* and *right* DataFrames need to be adjusted, ensuring these observations will be included in both of these DataFrames. As a result, the *matches* DataFrame must be expanded with an additional set of corresponding entries (highlighted by the <span style="color: orange;">orange</span> rows):

| Left | Right |
|------|-------|
| A    | B     |
| A    | C     |
| A    | D     |
| B    | C     |
| B    | D     |
| C    | D     |
| <span style="color: orange;">B</span>    | <span style="color: orange;">B</span>     | 
| <span style="color: orange;">C</span>    | <span style="color: orange;">C</span>     | 

This example highlights why a naive approach (merely adding an extra match) does not fully capture the nature of the linking problem.

### 5. Correcting the Relationships

To resolve this issue and correctly group all records representing the same real-world entity, we use the `data_preparation_cs` method from the `SetupData` class in the `neer_match_utilities.panel` module. This method automatically completes the matching pairs and adjusts the `left` and `right` datasets accordingly.


In [ ]:
from neer_match_utilities.panel import SetupData

left, right, matches = SetupData(matches=matches).data_preparation_cs(
    df_left=left,
    df_right=right,
    unique_id='company_id'
)

### 6. Verifying the Adjustments

Finally, we verify that the adjustments correctly reflect the intended relationships by checking the relevant company IDs in the updated datasets.


In [ ]:
# Verify the updated matches for the specific company_ids

artificial_group = ['1e87fc75b4', '810c9c3435', '0008e07878', '8bf51ba8a0']

matches_subset = matches[
    matches['left'].isin(artificial_group) |
    matches['right'].isin(artificial_group)
].sort_values(['left', 'right'])

matches_subset

In [ ]:
# Check the corresponding records in the left dataset

left_subset = left[
    left['company_id'].isin(artificial_group)
][['company_id']]
left_subset.head(10)

In [ ]:
# Check the corresponding records in the right dataset

right_subset = right[
    right['company_id'].isin(artificial_group)
][['company_id']]
right_subset

By following these steps, we ensure that the data accurately represents the underlying real-world relationships, even when the matching is more complex than a simple 1:1 mapping. To not have the manual change affect the next steps, we drop observations associated with these IDs.


In [ ]:
left = left[~left['company_id'].isin(artificial_group)].reset_index(drop=False)
right = right[~right['company_id'].isin(artificial_group)].reset_index(drop=False)
matches = matches[
    (~matches['left'].isin(artificial_group))
    &
    (~matches['right'].isin(artificial_group))
].reset_index(drop=False)

---

## Formatting

### 1. A customized `similarity_map`

Set up the `similarity_map`. Note that the columns as `city`, `industry`, and `purpose` contain missing values. One way to improve the handling of these is to include a custom [similarity function](https://github.com/maliedvp/py-neer-match/blob/custom_similarity_functions/src/neer_match/similarity_map.py) `notmissing` to the `similarity_map` that returns 0 if a least one observation of a record pair is any missing value (`None`, `np.nan`, `pd.nan` or and empty string) and 1 otherwise. Similarly, for numeric columns, the custom function `notzero` is added. These functions are not part of the released version of `neer_match.similarity_map.available_similarities()`, which is why they are outcommented in the example below.


In [ ]:
from neer_match.similarity_map import SimilarityMap
from neer_match_utilities.custom_similarities import CustomSimilarities

# Load custom similarities dedicated to missing values from the utilities package

CustomSimilarities()

# Define similarity_map

similarity_map = {
    "company_name" : [
        "levenshtein",
        "jaro_winkler",
        "prefix",
        "postfix",
        "token_sort_ratio",
        "token_set_ratio",
        "partial_token_set_ratio",
        "partial_token_sort_ratio",
    ],
    "city" : [
        "levenshtein",
        "jaro_winkler",
        "notmissing",
    ],
    "industry" : [
        "levenshtein",
        "jaro_winkler",
        "notmissing",
    ],
    "purpose" : [
        "levenshtein",
        "jaro_winkler",
        "token_sort_ratio",
        "token_set_ratio",
        "partial_token_set_ratio",
        "partial_token_sort_ratio",
        "notmissing",
    ],
    "bs_text" : [
        "levenshtein",
        "jaro_winkler",
        "token_sort_ratio",
        "token_set_ratio",
        "partial_token_set_ratio",
        "partial_token_sort_ratio",
        "notmissing",
    ],
    "found_year" : [
        "discrete",
        "notzero"
    ],
    "found_date_modified" : [
        "discrete",
        "notmissing",
    ]
}

smap = SimilarityMap(similarity_map)

### 2. Harmonizing the data

Next, data formatting can be harmonized using the `Prepare` class. This class enables operations such as capitalizing string variables and converting other values to numeric types. Importantly, these operations are applied consistently to both the *left* and *right* DataFrames.


In [ ]:
from neer_match_utilities.prepare import Prepare

# Initialize the Prepare object

prepare = Prepare(
    similarity_map=similarity_map, 
    df_left=left, 
    df_right=right, 
    id_left='company_id', 
    id_right='company_id',
    spacy_pipeline='de_core_news_sm',
    additional_stop_words=[]
)

# Get formatted and harmonized datasets

left, right = prepare.format(
    fill_numeric_na=False,
    to_numeric=['found_year'],
    fill_string_na=True, 
    capitalize=True,
    lower_case=False,
    remove_stop_words=False,
)

In [ ]:
left.head()

In [ ]:
right.head()

## Re-Structuring 

`neer-match` requires that the *matches* DataFrame be structured with the indices from the left and right datasets instead of their unique IDs. To convert your *matches* DataFrame into the required format, you can run:


In [ ]:
from neer_match_utilities.training import Training

training = Training(
    similarity_map=similarity_map, 
    df_left=left, 
    df_right=right, 
    id_left='company_id', 
    id_right='company_id'
)

matches = training.matches_reorder(
    matches, 
    matches_id_left='left', 
    matches_id_right='right'
)

matches.head()

Let's track down the observations from *matches* in *left* .


In [ ]:
left_index = matches.loc[4,'left']
left[left.index==left_index]

and *right*


In [ ]:
right_index = matches.loc[4,'right']
right[right.index==right_index]

## Splitting data

Subsequently, we need to split the data into training and test sets, each consisting of three DataFrames. The training ratio is given by $\text{training_ratio} = 1 - (\text{test_ratio} + \text{validation_ratio})$. Note that since validation is not implemented yet, you can set $\text{validation_ratio} = 0$.


In [ ]:
from neer_match_utilities.split import split_test_train

left_train, right_train, matches_train, left_validation, right_validation, matches_validation, left_test, right_test, matches_test = split_test_train(
    left = left,
    right = right,
    matches = matches,
    test_ratio = .5,
    validation_ratio = .0
)

In [ ]:
matches_train.head()

In [ ]:
left_train[
    left_train.index.isin(
        matches_train['left'].head()
    )
]

In [ ]:
right_train[
    right_train.index.isin(
        matches_train['right'].head()
    )
]